#### How to choose the granularity of the context such that the agent can choose spans

In [1]:
import nltk
import spacy
import benepar

from datasets import load_dataset

benepar.download('benepar_en3')
squad = load_dataset("readerbench/squad_qgen", split="train")

/opt/homebrew/anaconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package benepar_en3 to
[nltk_data]     /Users/teoneacsu/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
Using custom data configuration readerbench--squad_qgen-afbe6c1a706ab455
Found cached dataset parquet (/Users/teoneacsu/.cache/huggingface/datasets/readerbench___parquet/readerbench--squad_qgen-afbe6c1a706ab455/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [2]:
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

In [4]:
# generate a dataframe with sentence and all possible answers
import pandas as pd
squad_df = []

from tqdm import tqdm
# iterate through the dataset
for i, sample in tqdm(enumerate(squad), total=len(squad)):

    # eliminate the contexts that have sentences longer than 512 tokens
    try:
        doc = nlp(sample['context'])
    except:
        continue

    for sent in doc.sents:

        sent_text = sent.text

        constit_list = list(sent._.constituents)

        tag_constit_list = [(constit._.labels, constit.text.strip()) for constit in constit_list]

        sample['possible_answers'] = tag_constit_list

        squad_df.append((sent_text, tag_constit_list))

  0%|          | 0/82900 [00:00<?, ?it/s]/opt/homebrew/anaconda3/envs/ml/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/homebrew/anaconda3/envs/ml/lib/python3.10/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
 47%|████▋     | 38897/82900 [5:14:28<2:24:18,  5.08it/s]    

In [ ]:
squad_df_new = pd.DataFrame(squad_df, columns=['sentence', 'spans'])
squad_df_new.head()


,sentence,spans
0,The Normans (Norman: Nourmands; French: Norman...,"[((S,), The Normans (Norman: Nourmands; French..."
1,"They were descended from Norse (""Norman"" comes...","[((S,), They were descended from Norse (""Norma..."
2,Through generations of assimilation and mixing...,"[((S,), Through generations of assimilation an..."
3,The distinct cultural and ethnic identity of t...,"[((S,), The distinct cultural and ethnic ident..."
4,The Normans (Norman: Nourmands; French: Norman...,"[((S,), The Normans (Norman: Nourmands; French..."


In [ ]:
squad_df_new.to_json('squad_df_train.json')